In [1]:
%load_ext rich
from pathlib import Path
from corpus_base import settings, setup_base_tbls

Path().cwd()

PosixPath('/Users/mv/Code/corpus-base')

In [2]:
case = next(settings.case_folders)

In [3]:
case

PosixPath('/Users/mv/code/corpus/decisions/sc/38254/details.yaml')

In [4]:
from corpus_base import DecisionRow
obj = DecisionRow.from_path(case)

In [5]:
obj.id

'38254'

In [6]:
from corpus_base import settings

sql = settings.base_env.get_template("update_justice_ids.sql").render(
            justice_tbl=settings.JusticeTableName,
            decision_tbl=settings.DecisionTableName,
            target_decision_id=obj.id,
        )

In [7]:
from rich.syntax import Syntax
Syntax(sql, lexer="sql")

WITH candidate_justices AS (                                                                                       
    SELECT                                                                                                         
        jt.id,                                                                                                     
        jt.alias,                                                                                                  
        jt.last_name,                                                                                              
        COUNT(*) num                                                                                               
    FROM                                                                                                           
        justices_tbl                                                                                               
        jt                                                                                                         
    WHERE                                                                                                          
        jt.inactive_date > d.date -- decision written before the justice became inactive                           
        AND d.date > jt.start_term -- decision written after the justice was appointed                             
        AND d.per_curiam != 1                                                                                      
        AND d.raw_ponente IS NOT NULL                                                                              
        AND -- the decision raw ponente matches either:                                                            
        (                                                                                                          
            d.raw_ponente = alias -- 1. the alias (already lower case); or                                         
            OR d.raw_ponente = LOWER(                                                                              
                jt.last_name                                                                                       
            ) -- 2.  the cleaned lower-cased last name of the justice                                              
        )                                                                                                          
    GROUP BY                                                                                                       
        jt.id                                                                                                      
    HAVING                                                                                                         
        num = 1 -- ensure that only one justice id matches the criteria                                            
),                                                                                                                 
justices_matched(                                                                                                  
    decision_id,                                                                                                   
    matched_justice_id                                                                                             
) AS (                                                                                                             
    SELECT                                                                                                         
        d.id,                                                                                                      
        -- the decision id                                                                                         
        (                                                                                                          
            SELECT                                      

In [ ]:

settings.db.execute(sql)

In [1]:
from corpus_base import init
init(test_only=10)